In [2]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import DirectoryLoader

from anthropic import Anthropic

In [12]:
load_dotenv()

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

## Load data

In [98]:
loader = DirectoryLoader("../data/raw", glob="*.txt")
documents = loader.load()

for num, doc in enumerate(documents):
    print(f"Index: {num}, File name: {doc.metadata['source'][-15:]}, File length: {len(doc.page_content)}")

[nltk_data] Downloading package punkt to /Users/skv/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/skv/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Index: 0, File name: interview_3.txt, File length: 37995
Index: 1, File name: interview_2.txt, File length: 33951
Index: 2, File name: interview_1.txt, File length: 36025
Index: 3, File name: interview_5.txt, File length: 50584
Index: 4, File name: interview_4.txt, File length: 39364


In [101]:
interview_1 = documents[0].page_content
interview_2 = documents[1].page_content
interview_3 = documents[2].page_content
interview_4 = documents[3].page_content
interview_5 = documents[4].page_content

In [15]:
llm = Anthropic(api_key=ANTHROPIC_API_KEY)

In [49]:
def get_llm_results(temperature: float = 0.5, content: str=None):
    message = llm.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=4096,
        temperature=0.5,
        # system="",
        messages=[
            {"role": "user", "content": f"{content} + \nGive the final answer only in Russian "}
        ]
    )

    return message

In [103]:
# Structuring the transcripts:
question_1 = f"""
Here are some recordings of the interview: <interview>{interview_1}</interview> and <interview>{interview_2}</interview> 

Please make independent conclusions for each interview on:
1. Identify the main sections or phases of the interview, for example, introduction, general questions, main topics, concluding remarks. (In <main_sections> tags.)
2. Break down each transcript into these sections, marking the beginning and end of each part. (In <transcript_sections> tags.)
3. Create a standardized template for  <main_sections>structuring</main_sections> the transcripts consistently. (In <template_sections> tags.)
"""

answer_1 = get_llm_results(temperature=0.0, content=question_1)
print(answer_1.content[0].text)

<main_sections>
1. Введение и общие вопросы
2. Обучение и развитие навыков
3. Источники информации и контент
4. Заключительные вопросы
</main_sections>

<transcript_sections>
Первое интервью:
1. Введение и общие вопросы: "Давайте начнём тогда с таких общих вопросов."
2. Обучение и развитие навыков: "Так, хорошо, отлично, смотрите, раз вы новичок, да, в этой сфере, расскажите, пожалуйста, проходили ли вы какое-то обучение, ну, скажем так, в самом начале карьеры?"
3. Источники информации и контент: "Например, себе попробуйте, пожалуйста, вот сейчас записать ваши обязанности, да, основные, которые вы выполняете на той позиции, на которую перешли вот недавно."
4. Заключительные вопросы: "Так, Анастасия, спасибо вам большое за участие, было очень приятно с вами пообщаться, надеюсь, что вы не зря провели целый час, да, практически."

Второе интервью:
1. Введение и общие вопросы: "Давайте начнем тогда, наверное, с таких вот общих вопросов."
2. Обучение и развитие навыков: "Конечно, обязательн

In [70]:
# Performing initial coding:
question_2 = f"""
Here are some recordings of the interview: <interview>{interview_1}</interview> and <interview>{interview_2}</interview> 

Please make independent conclusions for each interview on:
1. Read through each interviews carefully and highlight or underline key phrases, words, or sentences that capture important ideas, concepts, or experiences. (In <key_phrases_sections> tags.)
2. Assign preliminary codes or labels to these highlighted segments, using short descriptive phrases or words that summarize the main idea. (In <code_sections> tags.)
"""

# answer_2 = get_llm_results(temperature=0.0, content=question_2)
print(answer_2.content[0].text)

<interview1>
<key_phrases_sections>
- Светлана, 50 лет, живет в Москве, работает риелтором с 2015 года
- Образование у нее педагогическое, но сейчас риелторство является ее основной деятельностью
- Занимается полным спектром сделок: аренда, вторичный рынок, новостройки
- Больше сделок по подбору, чем по продаже вторичной недвижимости
- В основном офлайн-сделки, онлайн-сделки по новостройкам не так много
- В обязанности входит обучение сотрудников, помощь в ведении сделок, взаимодействие с партнерами
- Проходила обязательное обучение в учебном центре агентства, где изучали продажи, психотипы клиентов, юридические вопросы
</key_phrases_sections>

<code_sections>
- Профиль респондента
- Сферы деятельности
- Распределение сделок
- Обязанности
- Обучение в начале карьеры
</code_sections>

<interview2>
<key_phrases_sections>
- Элла, 42 года, частный риелтор, работает около 5-7 лет
- Пришла к риелторству после необходимости продать свою квартиру
- Работает в основном с вторичным рынком жилья,

In [71]:
interview_code_1 = "Профиль респондента, Сферы деятельности, Распределение сделок, Обязанности, Обучение в начале карьеры"
interview_code_2 = "Профиль респондента, Приход в профессию, Специализация, Обязанности, Обучение в начале карьеры, Отношение к обучению, Важность hard и soft skills"

In [75]:
# Finding topics and themes:
question_3 = f"""
Here are some interview recordings and thematic codes for them: 
<interview_1>{interview_1}</interview_1> and <interview_code_1>{interview_code_1}</interview_code_1>,
<interview_2>{interview_2}</interview_2> and <interview_code_2>{interview_code_2}</interview_code_2>.


Please make independent conclusions for each interview on:
1. Look through the codes (<interview_code_1>, <interview_code_2>) and find patterns, similarities, or relationships among them. (In <relationships_sections> tags.)
2. Group related codes together into broader categories or themes that capture the main topics discussed in the interviews. (In <group_sections> tags.)
3. Identify any sub-themes within each main theme to provide a more detailed understanding of the topics. (In <sub-themes_sections> tags.)
4. Create a visual representation (e.g., mind map, thematic network) to illustrate the relationships between the themes and sub-themes. (In <map_sections> tags.)
"""

# answer_3 = get_llm_results(temperature=0.0, content=question_3)
print(answer_3.content[0].text)

<relationships_sections>
Интервью 1 и 2 имеют ряд общих тем и паттернов:

1. Профиль респондента: в обоих интервью обсуждается возраст, место жительства и опыт работы респондентов в сфере риелторства.

2. Приход в профессию: в интервью 2 подробно рассказывается, как респондент пришел в риелторство, в то время как в интервью 1 этот аспект затрагивается более кратко.

3. Специализация: в обоих интервью обсуждаются виды сделок, с которыми работают респонденты, с акцентом на вторичном и первичном рынке недвижимости.

4. Обязанности: в интервью 1 и 2 затрагиваются должностные обязанности респондентов, включая взаимодействие с клиентами, партнерами и сотрудниками.

5. Обучение в начале карьеры: в обоих интервью обсуждается прохождение обязательного обучения в начале карьеры, его формат и содержание.

6. Отношение к обучению: в интервью 2 более подробно раскрывается отношение респондента к обучению, его предпочтения в форматах и тематиках.

7. Важность hard и soft skills: в интервью 2 подробн

In [79]:
# Calculating the frequency of problems mentioned
question_4 = f"""
Here are some recordings of the interview: <interview>{interview_1}</interview> and <interview>{interview_2}</interview> 

Please make independent conclusions for each interview on:
1. Make a list of all the specific problems or issues mentioned by the interviewees. (In <problem_sections> tags.)
2. Go through each interview and tally the number of times each problem is mentioned. (In <problem_count_sections> tags.)
3. Calculate the frequency of each problem by dividing the number of mentions by the total number of interviews. (In <problem_total_count_sections> tags.)
4. Create a table or chart to display the frequency of each problem in descending order. (In <problem_total_table_sections> tags.)
"""

# answer_4 = get_llm_results(temperature=0.0, content=question_4)
print(answer_4.content[0].text)

<problem_sections>
- Необходимость обучения и повышения квалификации для риелторов
- Сложности в поиске качественной информации и обучающих материалов
- Изменения на рынке недвижимости, в том числе рост популярности первичного жилья
- Необходимость развития как hard, так и soft skills
- Проблемы с коммуникацией и привлечением клиентов в современных условиях
- Недостаток информации по инвестициям в недвижимость
- Недостаток информации по юридическим аспектам в риелторской деятельности
</problem_sections>

<problem_count_sections>
- Необходимость обучения и повышения квалификации для риелторов: 2
- Сложности в поиске качественной информации и обучающих материалов: 2
- Изменения на рынке недвижимости, в том числе рост популярности первичного жилья: 2
- Необходимость развития как hard, так и soft skills: 2
- Проблемы с коммуникацией и привлечением клиентов в современных условиях: 2
- Недостаток информации по инвестициям в недвижимость: 2
- Недостаток информации по юридическим аспектам в ри

In [82]:
problems = "Необходимость обучения и повышения квалификации для риелторов, Сложности в поиске качественной информации и обучающих материалов, Изменения на рынке недвижимости, в том числе рост популярности первичного жилья, Необходимость развития как hard, так и soft skills ,Проблемы с коммуникацией и привлечением клиентов в современных условиях, Недостаток информации по инвестициям в недвижимость, Недостаток информации по юридическим аспектам в риелторской деятельности"

In [87]:
# Grouping problems into categories
question_5 = f"""
Here are some interview recordings and common problems that were identified by all respondents in these interviews:
<interview>{interview_1}</interview> and <interview>{interview_2}</interview> and <problems>{problems}</problems> 

Please make independent conclusions for each interview on:
1. Review the list of problems (<problems>) and look for commonalities or shared characteristics among them. (In <common_features_sections> tags.)
2. Develop a set of categories or problem (<problems>) types that can encompass the various issues mentioned. (In <problems_categories_sections> tags.)
3. Assignment each problem (<problems>) to one or more relevant categories. (In <map_sections> tags.)
4. Create a summary table or matrix showing the problems grouped by category. (In <results_sections> tags.)
"""

answer_5 = get_llm_results(temperature=0.0, content=question_5)
print(answer_5.content[0].text)

<common_features_sections>
На основе анализа интервью и списка проблем, были выявлены следующие общие черты:

1. Необходимость постоянного обучения и повышения квалификации для риелторов в связи с изменениями на рынке недвижимости.
2. Сложности в поиске качественной и актуальной информации, обучающих материалов и экспертных знаний.
3. Изменения в структуре и поведении рынка, в частности, рост популярности первичного жилья и снижение роли риелторов.
4. Необходимость развития как профессиональных (hard skills), так и коммуникативных (soft skills) навыков.
5. Проблемы с привлечением и взаимодействием с клиентами в современных условиях.
6. Недостаток информации по отдельным аспектам риелторской деятельности, таким как инвестиции в недвижимость и юридические вопросы.
</common_features_sections>

<problems_categories_sections>
На основе выявленных общих черт можно выделить следующие категории проблем:

1. Необходимость непрерывного обучения и повышения квалификации
2. Поиск и доступ к качест

In [89]:
problems_categories = "Необходимость непрерывного обучения и повышения квалификации, Поиск и доступ к качественной профессиональной информации, Изменения на рынке недвижимости и адаптация к ним, Развитие hard и soft skills, Привлечение и взаимодействие с клиентами, Недостаток информации по специализированным вопросам"

In [97]:
# Making insights and forming hypotheses
question_6 = f"""
Here are some interview recordings and common problems that were identified by all respondents in these interviews:
<interview>{interview_1}</interview> and <interview>{interview_2}</interview> and <problems>{problems_categories}</problems> 

Please make final conclusions on all interviews at once, on the following points:
1. Reflect on the topics, frequency of problems, and categories of problems (<problems>). (In <reflection_sections> tags.) 
2. Analyze what these <reflection_sections>results</reflection_sections> say about the experiences, views, or difficulties of the interviewees. (In <results_sections> tags.) 
3. Pay attention to any unexpected patterns, connections, or contrasts in the data. Share these observations. (In <unexpected_sections> tags.)
4. Develop preliminary explanations or hypotheses about why certain topics or issues have arisen and how they may relate to each other or to broader contextual factors. (In <hypotheses_sections> tags.)
5. Consider alternative explanations and counterexamples to clarify your <hypotheses_sections>hypothesis</hypotheses_sections>.  (In <alternative_sections> tags.)
6. Identify areas where further research or analysis may be needed to validate or validate your <alternative_sections>ideas</alternative_sections>. (In <further_research_sections> tags.)
"""

answer_6 = get_llm_results(temperature=0.5, content=question_6)
print(answer_6.content[0].text)

<reflection_sections>
Анализируя интервью, можно выделить следующие основные проблемы, с которыми сталкиваются риэлторы:

1. Необходимость непрерывного обучения и повышения квалификации. Это связано с постоянными изменениями на рынке недвижимости, появлением новых инструментов, законодательными изменениями и т.д. Риэлторы должны быстро адаптироваться к этим изменениям.

2. Поиск и доступ к качественной профессиональной информации. Риэлторы отмечают, что часто сталкиваются с проблемой получения актуальной, достоверной и полезной информации по их сфере деятельности.

3. Изменения на рынке недвижимости и адаптация к ним. Рынок недвижимости постоянно меняется, и риэлторам необходимо быстро реагировать на эти изменения, перестраивать свои бизнес-процессы.

4. Развитие hard и soft skills. Риэлторам нужно сочетать профессиональные знания и навыки (hard skills) с коммуникативными и клиентоориентированными навыками (soft skills).

5. Привлечение и взаимодействие с клиентами. Риэлторы сталкивают